In [3]:
!pip install -q langchain pymilvus ollama pypdf langchainhub langchain-community langchain-experimental langchain-milvus


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Reshimming asdf python...


In [2]:
#from pymilvus import connections, Collection, MilvusClient, Milvus
from langchain_milvus import Milvus
from pymilvus import MilvusClient
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from langchain.embeddings.base import Embeddings

class BGEMilvusEmbeddings(Embeddings):

    def __init__(self):
        self.model = BGEM3EmbeddingFunction(
            model_name='BAAI/bge-m3',
            device='cpu',
            use_fp16=False #set to false if device='cpu'
        )

    def embed_documents(self, texts):
        embeddings = self.model.encode_documents(texts)
        return [i.tolist() for i in embeddings["dense"]]

    def embed_query(self, text):
        embedding = self.model.encode_queries([text])
        return embedding["dense"][0].tolist()

#ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")

embedding_model = BGEMilvusEmbeddings()


MILVUS_URL = "../LegalDefAgent/vec_db/definitions_vectors_refs.db"#"http://localhost:19530"
COLLECTION_NAME = 'Definitions'

#milvus_client = MilvusClient(uri=MILVUS_URL)
vector_store = Milvus(
    embedding_function=embedding_model,
    connection_args={"uri": MILVUS_URL},
    collection_name=COLLECTION_NAME,
    vector_field="dense_vector",
    text_field="definition_text"
)


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
results = vector_store.similarity_search(
    "operating agreement",
    k=10,
)
results

[Document(metadata={'id': 454042227454788615}, page_content='operation: means a project, contract, action or group of projects selected by the joint monitoring committee or the contracting authority of the programme concerned, or under its responsibility, that contribute to the objectives, as regards the cross-border cooperation programmes under point (a) of Article 27, of the priority axis or priority axes to which it relates or, as regards cross-border cooperation programmes in points (b) or (c) of Article 27 of a thematic priority or thematic priorities to which it relates;'),
 Document(metadata={'id': 454042226712654659}, page_content='operating site: means a site selected by the operator or pilot-in-command for landing, take-off and/or hoist operations;'),
 Document(metadata={'id': 454042228231263740}, page_content='interconnection agreement: means an agreement entered into by adjacent transmission system operators, whose systems are connected at a particular interconnection point

In [53]:
from langchain_ollama import OllamaLLM
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OllamaLLM(
    model="phi3:latest",
    callback_manager=CallbackManager(
        [StreamingStdOutCallbackHandler()]
    ),
    stop=["<|eot_id|>"],
)

In [21]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.1:latest",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content='Here\'s the translation:\n\n"J\'adore la programmation."\n\n(Note: "programmation" is a more formal or technical term for programming, but if you want to be more informal and say "j\'adore coder", that would also work!)', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-11-19T17:34:57.142529427Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 46440404597, 'load_duration': 11997580522, 'prompt_eval_count': 32, 'prompt_eval_duration': 5740109000, 'eval_count': 54, 'eval_duration': 28549487000}, id='run-87e07f04-be2c-446a-973a-2e200e4cb16b-0', usage_metadata={'input_tokens': 32, 'output_tokens': 54, 'total_tokens': 86})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.1:latest")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

In [89]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


SYSTEM_PROMPT = """
Human: You are a legal AI assistant. You are able to find the correct legal definition of terms out of the ones provided.
"""
USER_PROMPT = """
Use the following definitions enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{query}
</question>
"""

prompt = PromptTemplate(
    template=USER_PROMPT,
    input_variables=['context', 'query']
)

retriever = vector_store.as_retriever(search_kwargs={'k': 4})

In [90]:
query = "What is the definition of a contract?"


rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | llm
    | StrOutputParser()
)

rag_chain.invoke({"question": query})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is the definition of a contract?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "What is the definition of a contract?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "What is the definition of a contract?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "question": "What is the definition of a contract?"
}
[chain/error] [chain:RunnableSequence > chain:RunnableParallel<context,question>] s] Chain run errored with error:
"ValueError('text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).')Traceback (most 

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [103]:
import langchain
langchain.debug = True

In [99]:
t = retriever.invoke(query)
docs = t

"\n\n".join(doc.page_content for doc in docs)

'vessel: means any type of water craft, including boats, dinghies, floating platforms, non-displacement craft and seaplanes, used or capable of being used at sea;\n\nvessel: means a housing designed and built to contain fluids under pressure including its direct attachments up to the coupling point connecting it to other equipment; a vessel may be composed of more than one chamber;\n\nvessel: means an inland waterway vessel or seagoing ship;\n\nreefer vessel: means a vessel equipped to store and transport palletized or loose cargo (bulk) goods in temperature controlled holds or chambers;'

In [100]:
retriever.invoke(query)

[Document(metadata={'id': 454042227591628569}, page_content='vessel: means any type of water craft, including boats, dinghies, floating platforms, non-displacement craft and seaplanes, used or capable of being used at sea;'),
 Document(metadata={'id': 454042227335774533}, page_content='vessel: means a housing designed and built to contain fluids under pressure including its direct attachments up to the coupling point connecting it to other equipment; a vessel may be composed of more than one chamber;'),
 Document(metadata={'id': 454042227979079724}, page_content='vessel: means an inland waterway vessel or seagoing ship;'),
 Document(metadata={'id': 454042228884792327}, page_content='reefer vessel: means a vessel equipped to store and transport palletized or loose cargo (bulk) goods in temperature controlled holds or chambers;')]

In [106]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful legal AI assistant that is able to find the correct legal definitions of terms out of the ones provided."
    "Pick one out of the definitions provided as context to answer the question."
    "If the context is empty, answer with 'I can't answer'."
)

human_prompt = HumanMessagePromptTemplate.from_template(
    """Context:

    {context}

    Question: {question}"""
)

def format_defs(defs):
    return "\n\n".join(definition.page_content for definition in defs)


def build_chain(retriever, model):
    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    model = OllamaLLM(
        model="phi3:latest",
    )

    # RAG pipeline
    chain = (
        {"context": retriever | format_defs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    return chain

retriever = vector_store.as_retriever(search_kwargs={'k': 4})

chain = build_chain(retriever, model)

result = chain.invoke(
    "What is the definition of a vessel in the context of inland waterways?"
)
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the definition of a vessel in the context of inland waterways?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What is the definition of a vessel in the context of inland waterways?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the definition of a vessel in the context of inland waterways?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What is the definition of a vessel in the context of inland waterways?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What is the definition of a v

In [ ]:


PROMPT_TEMPLATE = """
You are a legal AI assistant. You are able to find the correct legal definition of terms out of the ones provided.
Pick one out of the definitions enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
If the <context> element is empty, answer with "i don't know".
<context>{context}</context>
<question>{question}</question>
"""

query = "What is the definition of vessel?"

embeddings = BGEMilvusEmbeddings()
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context", "question"])


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

res = rag_chain.invoke(query)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

ConnectError: [Errno 111] Connection refused

In [96]:
retriever.invoke(query)

[Document(metadata={'id': 454042227591628569}, page_content='vessel: means any type of water craft, including boats, dinghies, floating platforms, non-displacement craft and seaplanes, used or capable of being used at sea;'),
 Document(metadata={'id': 454042227335774533}, page_content='vessel: means a housing designed and built to contain fluids under pressure including its direct attachments up to the coupling point connecting it to other equipment; a vessel may be composed of more than one chamber;'),
 Document(metadata={'id': 454042227979079724}, page_content='vessel: means an inland waterway vessel or seagoing ship;'),
 Document(metadata={'id': 454042228884792327}, page_content='reefer vessel: means a vessel equipped to store and transport palletized or loose cargo (bulk) goods in temperature controlled holds or chambers;')]

In [52]:
from langchain import hub
from langchain.chains import RetrievalQA

query = input("\nQuery: ")
prompt = hub.pull("rlm/rag-prompt")   
    
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_db.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

result = qa_chain({"query": query})
print(result)

NameError: name 'vector_db' is not defined